In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV ,cross_val_score 
from sklearn.metrics import accuracy_score, precision_score , confusion_matrix , roc_auc_score , roc_curve ,auc, recall_score
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import ListedColormap
from sklearn.svm import SVC

In [3]:
salary_test=pd.read_csv("C:\\Users\\prasa\\Downloads\\SalaryData_Test(1).csv")
salary_train=pd.read_csv("C:\\Users\\prasa\\Downloads\\SalaryData_Train(1).csv")

In [4]:
salary_test.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


In [5]:
salary_test.isnull().values.any()

False

In [6]:
salary_test.duplicated().values.any()

True

In [7]:
test=salary_test.drop_duplicates()

In [8]:
test.shape

(14130, 14)

In [9]:
salary_train.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [10]:
salary_train.isnull().values.any()

False

In [11]:
salary_train.duplicated().values.any()

True

In [12]:
train=salary_train.drop_duplicates()

In [13]:
train.shape

(26903, 14)

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in test.columns:
    print(i)
    if test[i].dtypes== object:
        test[i]=le.fit_transform(test[i])
    else:
        pass

age
workclass
education
educationno
maritalstatus
occupation
relationship
race
sex
capitalgain
capitalloss
hoursperweek
native
Salary


<ipython-input-14-de34b20e0131>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[i]=le.fit_transform(test[i])


In [15]:
test.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,2,1,7,4,6,3,2,1,0,0,40,37,0
1,38,2,11,9,2,4,0,4,1,0,0,50,37,0
2,28,1,7,12,2,10,0,4,1,0,0,40,37,1
3,44,2,15,10,2,6,0,2,1,7688,0,40,37,1
4,34,2,0,6,4,7,1,4,1,0,0,30,37,0


In [16]:
for i in train.columns:
    print(i)
    if train[i].dtypes== object:
        train[i]=le.fit_transform(train[i])
    else:
        pass

age
workclass
education
educationno
maritalstatus
occupation
relationship
race
sex
capitalgain
capitalloss
hoursperweek
native
Salary


<ipython-input-16-c08d68009528>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=le.fit_transform(train[i])


In [17]:
train.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,5,9,13,4,0,1,4,1,2174,0,40,37,0
1,50,4,9,13,2,3,0,4,1,0,0,13,37,0
2,38,2,11,9,0,5,1,4,1,0,0,40,37,0
3,53,2,1,7,2,5,0,2,1,0,0,40,37,0
4,28,2,9,13,2,9,5,2,0,0,0,40,4,0


In [18]:
# Model Building 
x_train = train.drop(labels='Salary',axis=1)
x_train.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native
0,39,5,9,13,4,0,1,4,1,2174,0,40,37
1,50,4,9,13,2,3,0,4,1,0,0,13,37
2,38,2,11,9,0,5,1,4,1,0,0,40,37
3,53,2,1,7,2,5,0,2,1,0,0,40,37
4,28,2,9,13,2,9,5,2,0,0,0,40,4


In [19]:
y_train = train['Salary']
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: Salary, dtype: int32

In [20]:
x_test = test.drop('Salary',axis=1)
x_test.head(5)

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native
0,25,2,1,7,4,6,3,2,1,0,0,40,37
1,38,2,11,9,2,4,0,4,1,0,0,50,37
2,28,1,7,12,2,10,0,4,1,0,0,40,37
3,44,2,15,10,2,6,0,2,1,7688,0,40,37
4,34,2,0,6,4,7,1,4,1,0,0,30,37


In [21]:
y_test = test['Salary']
y_test.head()

0    0
1    0
2    1
3    1
4    0
Name: Salary, dtype: int32

### Building SVM model with Hyper Parameters kernel= 'rbf'

In [22]:
model_rbf=SVC(kernel='rbf')
model_rbf.fit(x_train,y_train)
y_pred_rbf = model_rbf.predict(x_test)

In [23]:
accuracy_score(y_test,y_pred_rbf)

0.7960368011323425

### Building SVM Model with kernel= 'Poly'

In [24]:
model_poly=SVC(kernel='poly')
model_poly.fit(x_train,y_train)
y_pred_poly = model_poly.predict(x_test)

In [25]:
accuracy_score(y_test,y_pred_poly)

0.778414720452937

### Building SVM Model with kernel = 'Sigmoid'

In [26]:
model_sig=SVC(kernel='sigmoid')
model_sig.fit(x_train,y_train)
y_pred_sig = model_sig.predict(x_test)

In [28]:
accuracy_score(y_test,y_pred_sig)

0.7544232130219392

#### We got the best accuracy with RBF kernel